# 🦟 Tech Challenge - Sistema de Diagnóstico de Dengue com ML

## Objetivo
Desenvolver um modelo de Machine Learning capaz de classificar casos suspeitos de dengue em **Confirmado** ou **Descartado**, utilizando dados clínicos e epidemiológicos do SINAN (2022).

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, recall_score, f1_score, roc_auc_score, precision_score, roc_curve

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Desbalanceamento
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Interpretação
import shap

# Configurações
pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")
%matplotlib inline

## 2. Fase 1: Exploração de Dados
- Carregamento do dataset
- Análise inicial (Missing values, Distribuição de classes)

In [ ]:
# Carregar Dataset
filename = 'DENGBR22.csv'

if os.path.exists(filename):
    df = pd.read_csv(filename, low_memory=False)
    print(f"Dataset carregado com sucesso! Dimensões: {df.shape}")
else:
    print(f"❌ Erro: Arquivo '{filename}' não encontrado.")
    print(f"Diretório atual de trabalho: {os.getcwd()}")
    print("Arquivos no diretório atual:", os.listdir())
    # Levantar erro para parar a execução e não causar NameError depois
    raise FileNotFoundError(f"Arquivo {filename} não encontrado no diretório atual.")

In [ ]:
# Visualizar primeiras linhas
df.head()

In [ ]:
# Distribuição da variável alvo original
print("Distribuição original de CLASSI_FIN:")
print(df['CLASSI_FIN'].value_counts(dropna=False))

## 3. Fase 2: Pré-processamento e Feature Engineering

In [ ]:
# 1. Filtrar Classes Relevantes
# Manter apenas: 1 (Confirmado - Clássico), 2 (Descartado), 10 (Dengue), 11 (Dengue com sinais de alarme), 12 (Dengue grave)
# Obs: O dicionário pode variar. Assumindo mapeamento padrão do SINAN para 2022:
# Vamos focar em: 10, 11, 12 como POSITIVO e 5 como DESCARTADO (verificar dicionário exato)
# Baseado no planejamento anterior e prompt: 
# "Classificação Final: 1 (Confirmado) vs 2 (Descartado)" -> Vamos filtrar estes primeiro.

# *Ajuste conforme dicionário específico do ano se necessário*
# Para este exercício, seguiremos a instrução: "1 e 2"

df_clean = df[df['CLASSI_FIN'].isin([1, 2])].copy()
print(f"Registros após filtro de classe (1 ou 2): {df_clean.shape}")

# Mapear Target: 1 -> 1 (Positivo), 2 -> 0 (Negativo)
df_clean['target'] = df_clean['CLASSI_FIN'].map({1: 1, 2: 0})
print("Distribuição do Target:")
print(df_clean['target'].value_counts(normalize=True))

In [ ]:
# 2. Tratamento de Idade (NU_IDADE_N)
def tratar_idade(valor):
    valor_str = str(valor).zfill(4)
    unidade = int(valor_str[0])
    qtd = int(valor_str[1:])
    
    if unidade == 4: # Anos
        return qtd
    elif unidade == 3: # Meses
        return qtd / 12
    elif unidade == 2: # Dias
        return qtd / 365
    else:
        return np.nan

df_clean['idade_anos'] = df_clean['NU_IDADE_N'].apply(tratar_idade)
df_clean = df_clean.dropna(subset=['idade_anos']) # Remover idades inválidas

In [ ]:
# 3. Seleção de Features

features_sintomas = [
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 
    'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 
    'LEUCOPENIA', 'LACO', 'DOR_RETRO'
]

features_comorbidades = [
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 
    'ACIDO_PEPT', 'AUTO_IMUNE'
]

features_demograficas = ['CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N']

# Converter binários (1=Sim, 2=Não) -> (1, 0)
for col in features_sintomas + features_comorbidades:
    # Preencher NaN com 2 (Não) antes de converter, assumindo ausência de registro como negativo
    # Ou usar estratégia mais conservadora. Vamos preencher com 2.
    df_clean[col] = df_clean[col].fillna(2)
    df_clean[col] = df_clean[col].apply(lambda x: 1 if x == 1 else 0)

# Feature Engineering: Contagens
df_clean['qtd_sintomas'] = df_clean[features_sintomas].sum(axis=1)
df_clean['qtd_comorbidades'] = df_clean[features_comorbidades].sum(axis=1)

In [ ]:
# Tratamento de Dados Temporais
df_clean['DT_NOTIFIC'] = pd.to_datetime(df_clean['DT_NOTIFIC'], errors='coerce')
df_clean['mes_notificacao'] = df_clean['DT_NOTIFIC'].dt.month

In [ ]:
# Preparar X e y
cols_to_use = features_sintomas + features_comorbidades + ['idade_anos', 'qtd_sintomas', 'qtd_comorbidades', 'mes_notificacao']

X = df_clean[cols_to_use]
y = df_clean['target']

# Tratamento de Missings (Imputação simples para numéricas/restantes)
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

print("Shape final de X:", X.shape)

In [ ]:
# Correlação
plt.figure(figsize=(15, 10))
sns.heatmap(df_clean[cols_to_use + ['target']].corr(), annot=False, cmap='coolwarm')
plt.title("Matriz de Correlação")
plt.show()

## 4. Fase 3: Modelagem e Avaliação

In [ ]:
# Split Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

In [ ]:
# Pipelines de Modelos
# Usaremos scaler se necessário (p/ Regressão Logística)

models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'))
    ]),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss', scale_pos_weight=y_train.value_counts()[0]/y_train.value_counts()[1])
}

results = {}

for name, model in models.items():
    print(f"Treinando {name}...")
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    tp = recall_score(y_test, y_pred)
    fp = 1 - precision_score(y_test, y_pred, zero_division=0)
    
    results[name] = {
        'Recall': recall_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_proba),
        'Model': model
    }
    
    print(f"--- {name} ---")
    print(f"Recall: {results[name]['Recall']:.4f}")
    print(f"AUC-ROC: {results[name]['AUC-ROC']:.4f}\n")

In [ ]:
# Tabela Comparativa
df_res = pd.DataFrame(results).T.drop(columns=['Model'])
df_res[['Recall', 'Precision', 'F1-Score', 'AUC-ROC']].style.background_gradient(cmap='Greens')

## 5. Fase 5: Interpretação (SHAP e Importância)

In [ ]:
# Feature Importance (Random Forest)
rf_model = results['Random Forest']['Model']
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(15).plot(kind='barh', title='Top 15 Feature Importances (RF)')
plt.show()

In [ ]:
# SHAP Values (XGBoost) - Implementação Robusta
import shap

# Inicializar JS (necessário para alguns plots interativos no notebook)
shap.initjs()

try:
    xgb_model = results['XGBoost']['Model']
    
    # Usar uma amostra do teste para agilizar (SHAP pode ser lento)
    X_sample = X_test.iloc[:500]
    
    print("Calculando SHAP values...")
    
    # Tentar usar o TreeExplainer (específico para árvores)
    explainer = shap.TreeExplainer(xgb_model)
    
    # Calcular shape values
    # check_additivity=False pode ser necessário em alguns casos de instabilidade numérica
    shap_values = explainer.shap_values(X_sample, check_additivity=False)
    
    # Tratamento para outputs diferentes (algumas versões retornam lista para classificação)
    if isinstance(shap_values, list):
        print("SHAP retornou lista (provavelmente classes separadas). Usando classe positiva.")
        # Para binário, o índice 1 geralmente é a classe positiva
        shap_values_to_plot = shap_values[1]
    else:
        shap_values_to_plot = shap_values
        
    # Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values_to_plot, X_sample, show=True)

except Exception as e:
    print(f"Erro ao gerar análise SHAP: {e}")
    print("Dica: Verifique se o pacote 'shap' está instalado corretamente e se as versões são compatíveis.")

## Discussão dos Resultados

- **Recall:** Priorizamos esta métrica pois falsos negativos são perigosos no contexto de dengue.
- **Variáveis Importantes:** A análise SHAP mostra quais sintomas e comorbidades mais impactam o diagnóstico.